In [1]:
import sklearn.tree
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
features_pd = pd.read_csv('ring_features.csv.bz2', compression='bz2')
targets_pd = pd.read_csv('ring_targets.csv')

features = features_pd.as_matrix()[:,1:]
targets = targets_pd.as_matrix()[:,1:].flatten()
print(features.shape, targets.shape)

(214304, 38) (214304,)


In [5]:
tmask = np.random.choice([False,True], len(targets), p=[0.75, 0.25])
vmask = np.invert(tmask)
train_X = features[tmask]
train_Y = targets[tmask]
test_X = features[vmask]
test_Y = targets[vmask]

In [ ]:
dtrain = xgb.DMatrix(train_X, label=train_Y)
dtest = xgb.DMatrix(test_X, label=test_Y)

cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8,
              'objective': 'binary:logistic'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), cv_params, cv=5, n_jobs=-1)
optimized_GBM.fit(train_X, train_Y)

In [ ]:
bst.get_fscore()

In [ ]:
model = sklearn.tree.DecisionTreeClassifier(criterion='gini')
model.fit(train_X, train_Y)
model.score(train_X, train_Y)
prediction = model.predict(test_X)


In [ ]:
scale = pd.Series.from_csv('ring_max.csv').as_matrix()

for i,p in enumerate(prediction):
    if p == 1:
        f = test_X[i] * scale
        for j,v in enumerate(f):
            if v > 0:
                print(features_pd.columns[j], v)
        print('------------------------------')
    if i > 10:
        break